TASK 1 (trigger & monitoring)
=============================

**Description**
- Entrega un script SQL completo con las consultas desarrolladas.
- Junto con un documento donde muestres los resultados obtenidos (mediante capturas de pantalla) y agregues una breve interpretación o comentario para cada uno.
- Crea un trigger que registre en una tabla de monitoreo cada vez que un producto supere las **200.000 unidades vendidas acumuladas**.

- El trigger debe activarse después de insertar una nueva venta y registrar en la tabla:
  - **ID del producto**
  - **Nombre del producto**
  - **Cantidad total de unidades vendidas**
  - **Fecha en que se superó el umbral**


___

**Monitoring Table**

```
CREATE TABLE product_monitoring (
  monitor_id      INT AUTO_INCREMENT PRIMARY KEY,
  productid       INT       NOT NULL,
  productname     VARCHAR(100) NOT NULL,
  total_selled    INT       NOT NULL,
  threshold_date  DATETIME  NOT NULL
);
```
- Creamos la tabla product_monitoring para registrar los eventos generados por el Trigger.

![primer respuesta](images/sa_q1.png)
___

**Trigger Condition**

```
DELIMITER $$
CREATE TRIGGER acum_sale_over_200k
AFTER INSERT ON sales
FOR EACH ROW
BEGIN
  DECLARE cum_qty INT;
  SELECT SUM(quantity)
    INTO cum_qty
    FROM sales
   WHERE productid = NEW.productid;

  IF cum_qty > 200000 THEN
    INSERT INTO product_monitoring (
      productid,
      productname,
      total_selled,
      threshold_date
    )
    SELECT
      NEW.productid,
      p.productname,
      cum_qty,
      NOW()
    FROM products p
    WHERE p.productid = NEW.productid;
  END IF;
END$$
DELIMITER ;

``` 

- Este trigger arranca justo despues de cada venta nueva en la tabla sales. 
- Suma todas las unidades vendidas de ese mismo producto y, si el total supera 200000, mete un registro en product_monitoring con:
- id del producto
- nombre del producto
- cantidad total vendida
 -fecha y hora en que paso el umbral

___



TASK 2 (testing trigger)
=============================

**Description**
- Registra una venta correspondiente al vendedor con ID 9, al cliente con ID 84, del producto con ID 103, por una cantidad de 1.876 unidades y un valor de 1200 unidades.
- Consulta la tabla de monitoreo, toma captura de los resultados y realiza un análisis breve de lo ocurrido.



``` 
INSERT INTO sales (
  saleid,
  salepersonid,
  customerid,
  productid,
  quantity,
  discount,
  totalprice,
  saledate,
  transactionnumber
) VALUES (
  1000000,    -- saleid único de prueba
  9,          -- vendedor ID 9
  84,         -- cliente ID 84
  103,        -- producto ID 103
  1876,       -- cantidad
  0.00,       -- descuento
  1200.00,    -- totalprice
  NOW(),
  'TEST-103-09'
);
``` 

``` 
SELECT *
FROM product_monitoring
WHERE productid = 103
ORDER BY monitor_id DESC;
``` 
![segunda respuesta](images/sa_q2.png)

___


TASK 3 (optimization)
=============================

**Description**
- Selecciona dos consultas del avance 1 y crea los índices que consideres más adecuados para optimizar su ejecución.
- Prueba con índices individuales y compuestos, según la lógica de cada consulta. Luego, vuelve a ejecutar ambas consultas y compara los tiempos de ejecución antes y después de aplicar los índices. 
- Finalmente, describe brevemente el impacto que tuvieron los índices en el rendimiento y en qué tipo de columnas resultan más efectivos para este tipo de operaciones.


**Indices Creados**
- CREATE INDEX idx_sales_productid ON sales (productid);
```para acelerar el GROUP BY sobre productid y evitar escanear toda la tabla. ```

- CREATE INDEX idx_sales_prod_person ON sales (productid, salepersonid);
```para cubrir en un solo indice el agrupamiento por productid y salepersonid. ```

- CREATE INDEX idx_sales_prod_person_qty ON sales (productid, salepersonid, quantity);
```para incluir quantity en el mismo indice y asi evitar saltos extra a la tabla de datos PK en products y employees: para que los JOINs sean busquedas directas en arbol B sin full scan ```



```
WITH
  sells_by_prod AS (
    SELECT productid, SUM(quantity) AS total_selled
    FROM sales
    GROUP BY productid
  ),
  sells_by_person AS (
    SELECT productid, salepersonid, SUM(quantity) AS total_selled
    FROM sales
    GROUP BY productid, salepersonid
  ),
  ranking_sellers AS (
    SELECT productid, salepersonid, total_selled,
           ROW_NUMBER() OVER (PARTITION BY productid ORDER BY total_selled DESC) AS rn
    FROM sells_by_person
  )
SELECT
  p.productid,
  p.productname,
  sbp.total_selled,
  CONCAT(e.firstname,' ',e.lastname) AS topseller,
  rs.total_selled    AS sellerqty,
  ROUND(rs.total_selled / sbp.total_selled * 100, 2) AS sellerpct
FROM sells_by_prod sbp
JOIN ranking_sellers rs
  ON rs.productid = sbp.productid AND rs.rn = 1
JOIN products p
  ON p.productid = sbp.productid
JOIN employees e
  ON e.employeeid = rs.salepersonid
ORDER BY sbp.total_selled DESC
LIMIT 5;
```

ANTES (11.3 SEGUNDOS)

![tercer respuesta](images/sa_q3.png)


AHORA (6.3 SEGUNDOS)

![tercer respuesta](images/sa_q4.png)


```
WITH prod_totals AS (
  SELECT productid, SUM(quantity) AS total_selled
  FROM sales
  GROUP BY productid
),
prod_rank_category AS (
  SELECT
    pt.productid,
    pt.total_selled,
    p.categoryid,
    ROW_NUMBER() OVER (PARTITION BY p.categoryid ORDER BY pt.total_selled DESC) AS cat_rank
  FROM prod_totals pt
  JOIN products p
    ON p.productid = pt.productid
)
SELECT
  prc.productid,
  p.productname,
  prc.total_selled,
  prc.categoryid,
  c.categoryname,
  prc.cat_rank
FROM prod_rank_category prc
JOIN products p
  ON p.productid = prc.productid
JOIN categories c
  ON c.categoryid = prc.categoryid
ORDER BY prc.total_selled DESC
LIMIT 10;
```

ANTES

![cuarta respuesta](images/sa_q5.png)



AHORA

![cuarta respuesta](images/sa_q6.png)